Fazer median house income e cenas...

In [130]:
import pandas as pd
import matplotlib.pyplot as plt

In [131]:
df = pd.read_csv('../data/raw/House_prices_23.csv')
df = df.iloc[:, 2:]
df.head()

,Preço,Localização,Área Terreno (m2),Quartos,Casas de Banho,Área Útil (m2),Estacionamento,Certificado Energético,Piso,Elevador,Frações,Pisos,Área Bruta Construção (m2),Área Habitação (m2),Área Implantação (m2),Viabilidade Construção,Vedado
0,45.000,"Oliveira do Hospital e São Paio de Gramaços, C...",4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,565.000,"Catraia de S. Paio, Oliveira do Hospital",17473,8,8,290,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,199.000,"Santa Maria, São Pedro e Sobral da Lagoa, Leiria",150,2,1,97,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,135.000,"Fânzeres e São Pedro da Cova, Porto",NaN,3,2,85,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sob Consulta,"Carvalhal Benfeito, Leiria",1280,3,1,150,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
df.drop(index=df[df['Preço'] == 'Sob Consulta'].index, axis=0, inplace=True) # drop colunas com preço 'Sob Consulta'
df.drop(index=df.loc[df['Preço'].isna()].index, axis=0, inplace=True) # remover nan do preço
df.Preço = df.Preço.apply(lambda x: str(x).replace('.', '')).astype('int64')

df['Quartos'] = df['Quartos'].apply(lambda x: str(x).split('-')[1] if '-' in str(x) else x) 
df['Quartos'] = df['Quartos'].astype('Int64')

df['Casas de Banho'] = df['Casas de Banho'].astype('Int64')
# nova feature cidade
df['City'] = df['Localização'].apply(lambda x: x.split(',')[-1].strip())
# df.groupby(['City']).median().sort_values(by='City', ascending=False).head(10)

features = ['Preço', 'City', 'Quartos', 'Casas de Banho', 'Área Útil (m2)']
df = df[features].copy()

X = df.iloc[:, 1:]
y = df.iloc[:, 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14769 entries, 0 to 16622
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Preço           14769 non-null  int64 
 1   City            14769 non-null  object
 2   Quartos         13893 non-null  Int64 
 3   Casas de Banho  13397 non-null  Int64 
 4   Área Útil (m2)  13853 non-null  object
dtypes: Int64(2), int64(1), object(2)
memory usage: 721.1+ KB


In [133]:
df.groupby(['City']).median().sort_values(by='Preço', ascending=False)

C:\Users\paro\AppData\Local\Temp\ipykernel_20852\1698697398.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['City']).median().sort_values(by='Preço', ascending=False)


,Preço,Quartos,Casas de Banho
City,,,
Montemor-o-Novo,1188000.0,3.0,3.0
Machico,855000.0,6.0,6.0
Pombal,750000.0,4.0,1.0
Cascais,695000.0,3.0,2.0
Monchique,690000.0,2.0,5.0
...,...,...,...
Arganil,37500.0,2.5,1.0
Vila Velha de Rodão,30000.0,3.0,1.0
Castro Daire,28500.0,1.5,1.0


In [134]:
df.sort_values(['Preço'], ascending=False).head(10)

,Preço,City,Quartos,Casas de Banho,Área Útil (m2)
14944,14500000,Loulé,11,14,585
15040,5500000,Lisboa,5,5,630
4857,5000000,Lisboa,5,6,665
13998,5000000,Lisboa,9,8,426
8109,4800000,Lisboa,4,4,327
9223,4800000,Lisboa,27,7,808
3499,4500000,Faro,3,3,270
8110,4500000,Lisboa,4,4,306
5786,4400000,Lisboa,4,8,279
6957,4400000,Lisboa,6,<NA>,NaN


In [135]:
df.value_counts(['City']).sort_values(ascending=False).head(10)

City            
Porto               1975
Lisboa              1229
Braga                924
Santarém             736
Aveiro               664
Faro                 583
Viana do Castelo     481
Viseu                454
Setúbal              421
Guarda               351
dtype: int64

In [136]:
df.loc[df.City == 'Machico'] # Só há uma casa no Machico portanto a media e mediana é super alta
                             # estraga o groupby

,Preço,City,Quartos,Casas de Banho,Área Útil (m2)
1522,855000,Machico,6,6,190


In [137]:
df.City.values

array(['Coimbra', 'Oliveira do Hospital', 'Leiria', ..., 'Porto',
       'Lisboa', 'Aveiro'], dtype=object)

## EDA

## Preprocessing

In [139]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
import numpy as np


num_pipe = Pipeline([
    ('impute_num', SimpleImputer(strategy='median')),
    ('standardize', StandardScaler())
])

cat_pipe = Pipeline([
    ('imputer_cat', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessing = ColumnTransformer([
    ('num', num_pipe, make_column_selector(dtype_include=np.number)),
    ('cat', cat_pipe, make_column_selector(dtype_include=object))
])

Prepared = preprocessing.fit_transform(X)

In [140]:
preprocessing.get_feature_names_out().shape

(744,)

In [141]:
X = pd.DataFrame(Prepared.toarray(), columns=preprocessing.get_feature_names_out(), index=df.index)
X

,num__Quartos,num__Casas de Banho,cat__City_Abrantes,cat__City_Aguiar da Beira,cat__City_Albergaria-a-Velha,cat__City_Albufeira,cat__City_Alcanena,cat__City_Alcobaça,cat__City_Alcoutim,cat__City_Alcácer do Sal,...,cat__Área Útil (m2)_913,cat__Área Útil (m2)_92,cat__Área Útil (m2)_93,cat__Área Útil (m2)_94,cat__Área Útil (m2)_95,cat__Área Útil (m2)_950,cat__Área Útil (m2)_96,cat__Área Útil (m2)_97,cat__Área Útil (m2)_98,cat__Área Útil (m2)_99
0,0.063007,-0.11042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.650343,5.21896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.654460,-0.99865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.063007,-0.11042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.063007,0.77781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16617,0.063007,-1.88688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16618,0.063007,-1.88688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16620,-0.654460,-0.99865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16621,0.780474,-0.11042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
from sklearn.linear_model import LinearRegression
# from sklearn.metrics import neg_root_mean_squared_error
from sklearn.model_selection import cross_val_score

lin_reg = LinearRegression()
lin_reg_rmses = -cross_val_score(lin_reg, X, y,
                                scoring='neg_root_mean_squared_error', cv=10)

In [143]:
pd.Series(lin_reg_rmses).describe()

count    1.000000e+01
mean     8.110110e+15
std      1.796413e+16
min      2.390538e+05
25%      2.667994e+05
50%      3.269926e+14
75%      1.324556e+15
max      5.498714e+16
dtype: float64